In [21]:
import pandas as pd 

In [23]:
providers = pd.read_csv("C:/Users/absah/Desktop/Food Waste Management Project/datasets/providers_data.csv")
receivers = pd.read_csv("C:/Users/absah/Desktop/Food Waste Management Project/datasets/receivers_data.csv")
food_listings = pd.read_csv("C:/Users/absah/Desktop/Food Waste Management Project/datasets/food_listings_data.csv")
claims = pd.read_csv("C:/Users/absah/Desktop/Food Waste Management Project/datasets/claims_data.csv")

In [38]:
providers.isnull().sum()
receivers.isnull().sum()
food_listings.isnull().sum()
claims.isnull().sum()

Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64

In [9]:
import sqlite3
conn = sqlite3.connect('../db/food_waste.db') 
cursor = conn.cursor()



In [27]:
# providers
cursor.execute('''
CREATE TABLE IF NOT EXISTS providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
)
''')

# receivers
cursor.execute('''
CREATE TABLE IF NOT EXISTS receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
)
''')

# food listings
cursor.execute('''
CREATE TABLE IF NOT EXISTS food_listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date TEXT,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY(Provider_ID) REFERENCES providers(Provider_ID)
)
''')

# claims
cursor.execute('''
CREATE TABLE IF NOT EXISTS claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp TEXT,
    FOREIGN KEY(Food_ID) REFERENCES food_listings(Food_ID),
    FOREIGN KEY(Receiver_ID) REFERENCES receivers(Receiver_ID)
)
''')


conn.commit()




In [25]:
# inserting the Dataframe data  into sqlite db table
providers.to_sql('providers', conn, if_exists='replace', index=False)
receivers.to_sql('receivers', conn, if_exists='replace', index=False)
food_listings.to_sql('food_listings', conn, if_exists='replace', index=False)
claims.to_sql('claims', conn, if_exists='replace', index=False)


1000

In [33]:
#check

pd.read_sql("SELECT * FROM providers LIMIT 5", conn)


,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


# Food Providers & Receivers

#### How many food providers and receivers are there in each city?


In [47]:
# Providers count / city
query_providers = '''
SELECT City, COUNT(*) AS Provider_Count
FROM providers
GROUP BY City
ORDER BY Provider_Count DESC
'''

# Receivers count / city
query_receivers = '''
SELECT City, COUNT(*) AS Receiver_Count
FROM receivers
GROUP BY City
ORDER BY Receiver_Count DESC
'''

provider_counts = pd.read_sql(query_providers, conn)
receiver_counts = pd.read_sql(query_receivers, conn)


print("Providers in Each City:")
display(provider_counts)

print("Receivers in Each City:")
display(receiver_counts)


Providers in Each City:


,City,Provider_Count
0,South Christopherborough,3
1,New Carol,3
2,Williamview,2
3,West Lauraborough,2
4,West Christopher,2
...,...,...
958,Alexanderchester,1
959,Aguirreville,1
960,Adamsville,1
961,Adamsview,1


Receivers in Each City:


,City,Receiver_Count
0,New Christopher,3
1,Williamsfort,2
2,West Richard,2
3,West James,2
4,Stevenmouth,2
...,...,...
961,Alexanderbury,1
962,Aguilarstad,1
963,Aguilarbury,1
964,Adamland,1


#### 2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food?


In [53]:
query2 = '''
SELECT Provider_Type, SUM(Quantity) AS Total_Quantity_Donated
FROM food_listings
GROUP BY Provider_Type
ORDER BY Total_Quantity_Donated DESC
'''

provider_type_stats = pd.read_sql(query2, conn)
display(provider_type_stats)


,Provider_Type,Total_Quantity_Donated
0,Restaurant,6923
1,Supermarket,6696
2,Catering Service,6116
3,Grocery Store,6059


#### 3.What is the contact information of food providers in a specific city?


In [60]:

city_input = input("Enter a city to find food providers and their contact details: ")


query3 = f'''
SELECT Name, Contact, Address
FROM providers
WHERE City = ?
'''

provider_contacts = pd.read_sql(query3, conn, params=(city_input,))
display(provider_contacts)


Enter a city to find food providers and their contact details:  South Christopherborough


,Name,Contact,Address
0,"Harris, Perez and Willis",581-281-6112,"549 Moody Road Suite 873\nJacobshire, VT 80966"
1,Torres-Dunlap,057.342.1300x473,Unit 1623 Box 5952\nDPO AA 55518
2,Brown and Sons,(352)481-4738,"725 Garner Locks\nAndrewchester, MI 49979"


#### 4.Which receivers have claimed the most food?


In [63]:
query4 = '''
SELECT r.Name AS Receiver_Name, COUNT(c.Claim_ID) AS Total_Claims
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Name
ORDER BY Total_Claims DESC
'''

top_receivers = pd.read_sql(query4, conn)
display(top_receivers)


,Receiver_Name,Total_Claims
0,William Frederick,5
1,Scott Hunter,5
2,Matthew Webb,5
3,Anthony Garcia,5
4,Kristine Martin,4
...,...,...
615,Alex Leon,1
616,Alan Campbell,1
617,Adam Browning,1
618,Aaron Scott,1


# Food Listings & Availability


#### 5.What is the total quantity of food available from all providers?


In [67]:
query5 = '''
SELECT SUM(Quantity) AS Total_Available_Quantity
FROM food_listings
'''

total_available = pd.read_sql(query5, conn)
display(total_available)


,Total_Available_Quantity
0,25794


#### 6.Which city has the highest number of food listings?


In [70]:
query6 = '''
SELECT Location AS City, COUNT(*) AS Number_of_Listings
FROM food_listings
GROUP BY Location
ORDER BY Number_of_Listings DESC
'''

top_listing_cities = pd.read_sql(query6, conn)
display(top_listing_cities)


,City,Number_of_Listings
0,South Kathryn,6
1,New Carol,6
2,Perezport,5
3,Jimmyberg,5
4,East Angela,5
...,...,...
619,Andersonmouth,1
620,Amandashire,1
621,Allenborough,1
622,Alexanderchester,1


#### 7.What are the most commonly available food types?


In [73]:
query7 = '''
SELECT Food_Type, COUNT(*) AS Count
FROM food_listings
GROUP BY Food_Type
ORDER BY Count DESC
'''

common_food_types = pd.read_sql(query7, conn)
display(common_food_types)


,Food_Type,Count
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


#### 8. How many food claims have been made for each food item?


In [76]:
query8 = '''
SELECT f.Food_Name, COUNT(c.Claim_ID) AS Total_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Food_Name
ORDER BY Total_Claims DESC
'''

food_claims = pd.read_sql(query8, conn)
display(food_claims)


,Food_Name,Total_Claims
0,Rice,122
1,Soup,114
2,Dairy,110
3,Fish,108
4,Salad,106
5,Chicken,102
6,Bread,94
7,Pasta,87
8,Vegetables,86
9,Fruits,71


#### 9. Which provider has had the highest number of successful food claims?


In [81]:
query9 = '''
SELECT p.Name AS Provider_Name, COUNT(c.Claim_ID) AS Successful_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN providers p ON f.Provider_ID = p.Provider_ID
WHERE c.Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC
'''

successful_providers = pd.read_sql(query9, conn)
display(successful_providers)


,Provider_Name,Successful_Claims
0,Barry Group,5
1,Miller Inc,4
2,"Harper, Blake and Alexander",4
3,Butler-Richardson,4
4,"Barnes, Castro and Curtis",4
...,...,...
243,"Anderson, Nicholson and Bruce",1
244,Allen-Mccullough,1
245,"Allen, Zimmerman and Harrington",1
246,Alexander PLC,1


#### 10. What percentage of food claims are completed vs. pending vs. canceled?


In [84]:
query10 = '''
SELECT Status,
       COUNT(*) AS Count,
       ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims), 2) AS Percentage
FROM claims
GROUP BY Status
'''

claim_status_distribution = pd.read_sql(query10, conn)
display(claim_status_distribution)


,Status,Count,Percentage
0,Cancelled,336,33.6
1,Completed,339,33.9
2,Pending,325,32.5


# Analysis & Insights


#### 11. What is the average quantity of food claimed per receiver?

In [90]:
query11 = '''
SELECT r.Name AS Receiver_Name,
       ROUND(AVG(f.Quantity), 2) AS Average_Quantity_Claimed
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Name
ORDER BY Average_Quantity_Claimed DESC
'''

avg_claim_per_receiver = pd.read_sql(query11, conn)
display(avg_claim_per_receiver)


,Receiver_Name,Average_Quantity_Claimed
0,Thomas Villanueva,50.0
1,Peggy Knight,50.0
2,Nancy Silva,50.0
3,Nancy Jones,50.0
4,Lisa Pitts,50.0
...,...,...
615,Kenneth Barnes,2.0
616,John Reynolds,2.0
617,Corey Rodriguez,2.0
618,Joshua Hooper,1.0


#### 12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

In [94]:
query12 = '''
SELECT f.Meal_Type, COUNT(c.Claim_ID) AS Claims_Count
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Meal_Type
ORDER BY Claims_Count DESC
'''

claimed_meals = pd.read_sql(query12, conn)
display(claimed_meals)


,Meal_Type,Claims_Count
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


#### 13.What is the total quantity of food donated by each provider?


In [97]:
query13 = '''
SELECT p.Name AS Provider_Name,
       SUM(f.Quantity) AS Total_Quantity_Donated
FROM food_listings f
JOIN providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.Name
ORDER BY Total_Quantity_Donated DESC
'''

donation_by_provider = pd.read_sql(query13, conn)
display(donation_by_provider)


,Provider_Name,Total_Quantity_Donated
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145
...,...,...
623,Reyes and Sons,1
624,Norris-Townsend,1
625,Mcgee PLC,1
626,"Martinez, Armstrong and Carroll",1
